<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/datasets/06_labeling_states.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06_labeling_states.ipynb — Rotulagem Supervisionada (BEFORE / DURING / AFTER)

Objetivo  
Formalizar o *ground truth* supervisionado em nível de janela (5 minutos), rotulando cada janela temporal como:

- **DURING**: janela pertencente a um episódio crítico detectado (Notebook 04)
- **BEFORE**: janela imediatamente anterior ao início de um episódio (janela de contexto)
- **AFTER**: janela imediatamente posterior ao fim de um episódio (janela de estabilização)
- **NORMAL**: janelas fora das três categorias acima (mantidas para completude, podendo ser filtradas em notebooks posteriores)

Entradas (artefatos do pipeline)

- `window_5min_features.parquet` (Notebook 05)
- `episodes_detected.parquet` (Notebook 04)

Saída (artefato deste notebook)

- `window_5min_labeled.parquet`

Estratégia de rotulagem

- A base temporal de referência é o eixo **bucket_id** (granularidade de 5 minutos).
- Para cada episódio \([start\_bucket, end\_bucket]\):
  - **DURING** recebe as janelas no intervalo fechado.
  - **BEFORE** recebe as *k* janelas imediatamente anteriores ao início do episódio.
  - **AFTER** recebe as *k* janelas imediatamente posteriores ao fim do episódio.
- Precedência (para evitar conflitos):
  1. DURING
  2. BEFORE / AFTER (não sobrescrevem DURING)
  3. NORMAL (default)

Parâmetros

- `K_BEFORE`: número de janelas de contexto anterior (default: 12 = 60 min)
- `K_AFTER`: número de janelas de estabilização posterior (default: 12 = 60 min)

Validações de consistência

- Checagem de integridade do eixo temporal (bucket_id ordenado)
- Checagem de sobreposição entre rótulos
- Distribuição final de classes
- Verificação de que todos os episódios têm DURING atribuído

Observações

- A presença de episódios muito próximos pode reduzir a quantidade efetiva de janelas BEFORE/AFTER (por conflito com DURING ou outro contexto).
- A coluna `state` gerada será usada como alvo supervisionado nos próximos notebooks (07_baseline_rf.ipynb).

In [1]:
# ============================================================
# 06_labeling_states.ipynb
# Rotulagem supervisionada BEFORE / DURING / AFTER (5-min)
# Pipeline PPCOMP_DM (Google Cluster Trace)
# ============================================================

# -----------------------------
# 0) BOOTSTRAP (Colab + Repo)
# -----------------------------

from pathlib import Path
import os
import sys
import subprocess
import importlib
import random
import numpy as np
import pandas as pd
import json

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Mount Google Drive (seguro)
if not Path("/content/drive/MyDrive").exists():
    from google.colab import drive
    drive.mount("/content/drive")
else:
    print("[Bootstrap] Google Drive já montado.")

# Ajuste aqui se o seu repo estiver em outro caminho do Drive
REPO_DIR = Path("/content/drive/MyDrive/Mestrado/PPCOMP_DM")
GITHUB_REPO = "https://github.com/sergiocostaifes/PPCOMP_DM.git"

if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    print(f"[Bootstrap] Clonando repositório em: {REPO_DIR}")
    subprocess.run(["git", "clone", GITHUB_REPO, str(REPO_DIR)], check=True)
else:
    # Atualiza repo sem quebrar notebook se houver alterações locais
    try:
        print("[Bootstrap] Atualizando repositório (git pull).")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)
    except Exception as e:
        print("[Bootstrap] Aviso: não foi possível atualizar via git pull:", e)

# Trabalhar sempre a partir do repo (evita path relativo quebrar)
os.chdir(str(REPO_DIR))
print("[Bootstrap] CWD =", os.getcwd())

# Garantir import do pacote src/
repo_str = str(REPO_DIR)
if repo_str not in sys.path:
    sys.path.insert(0, repo_str)

importlib.invalidate_caches()

# Paths padronizados
from src.paths import FEATURES_PATH, REPORTS_PATH, ensure_dirs
ensure_dirs()

print("FEATURES_PATH =", FEATURES_PATH)
print("REPORTS_PATH  =", REPORTS_PATH)

def log(msg: str) -> None:
    print(f"[06_labeling_states] {msg}")

# -----------------------------
# 1) INPUTS (Notebooks 04 e 05)
# -----------------------------

FEATURES_FILE = FEATURES_PATH / "window_5min_features.parquet"
EPISODES_FILE = FEATURES_PATH / "episodes_detected.parquet"

assert FEATURES_FILE.exists(), f"Arquivo não encontrado: {FEATURES_FILE}"
assert EPISODES_FILE.exists(), f"Arquivo não encontrado: {EPISODES_FILE}"

df = pd.read_parquet(FEATURES_FILE)
episodes = pd.read_parquet(EPISODES_FILE)

# Normalização mínima esperada neste pipeline (00–05 já usa bucket_id)
if "bucket_id" not in df.columns:
    raise KeyError(
        "Coluna 'bucket_id' não encontrada no dataset de features. "
        "Este notebook assume o eixo temporal do pipeline como bucket_id."
    )

required_ep_cols = {"start_bucket", "end_bucket"}
if not required_ep_cols.issubset(set(episodes.columns)):
    raise KeyError(
        f"Episódios sem colunas esperadas {required_ep_cols}. "
        f"Colunas disponíveis: {list(episodes.columns)}"
    )

df = df.sort_values("bucket_id").reset_index(drop=True)
episodes = episodes.sort_values(["start_bucket", "end_bucket"]).reset_index(drop=True)

log(f"Features: shape={df.shape}")
log(f"Episodes: shape={episodes.shape}")

# -----------------------------
# 2) Parâmetros de rotulagem
# -----------------------------

K_BEFORE = 12  # 12 * 5min = 60 min
K_AFTER  = 12  # 12 * 5min = 60 min

# -----------------------------
# 3) Inicializar rótulos
# -----------------------------

# state: NORMAL / BEFORE / DURING / AFTER
df["state"] = "NORMAL"

# Máscaras auxiliares
during_mask = np.zeros(len(df), dtype=bool)
before_mask = np.zeros(len(df), dtype=bool)
after_mask  = np.zeros(len(df), dtype=bool)

# Map rápido bucket_id -> índice (assume bucket_id único na série contínua)
# (03_window_5min_base.ipynb reindexa, então bucket_id tende a ser único)
bucket_to_idx = pd.Series(df.index.values, index=df["bucket_id"].values)

# -----------------------------
# 4) Rotular DURING por episódio
# -----------------------------

n_during_hits = 0
skipped_episodes = 0

for _, ep in episodes.iterrows():
    s = int(ep["start_bucket"])
    e = int(ep["end_bucket"])

    # Selecionar buckets no intervalo [s, e] que existam na série
    # (forma segura: filtra por condição, sem assumir contiguidade de índices)
    in_range = (df["bucket_id"] >= s) & (df["bucket_id"] <= e)
    hits = int(in_range.sum())
    if hits == 0:
        skipped_episodes += 1
        continue

    during_mask |= in_range.values
    n_during_hits += hits

log(f"DURING: janelas marcadas (com possíveis sobreposições por episódio): {n_during_hits}")
if skipped_episodes > 0:
    log(f"Aviso: episódios sem janelas correspondentes na série (skip): {skipped_episodes}")

# Aplicar DURING com maior precedência
df.loc[during_mask, "state"] = "DURING"

# -----------------------------
# 5) Rotular BEFORE e AFTER
# -----------------------------

# Para cada episódio, marcamos janelas BEFORE (s-K_BEFORE .. s-1) e AFTER (e+1 .. e+K_AFTER),
# mas sem sobrescrever DURING.
for _, ep in episodes.iterrows():
    s = int(ep["start_bucket"])
    e = int(ep["end_bucket"])

    # BEFORE
    before_start = s - K_BEFORE
    before_end   = s - 1
    if before_end >= before_start:
        in_before = (df["bucket_id"] >= before_start) & (df["bucket_id"] <= before_end)
        # não sobrescrever DURING
        in_before = in_before & (~during_mask)
        before_mask |= in_before.values

    # AFTER
    after_start = e + 1
    after_end   = e + K_AFTER
    if after_end >= after_start:
        in_after = (df["bucket_id"] >= after_start) & (df["bucket_id"] <= after_end)
        # não sobrescrever DURING
        in_after = in_after & (~during_mask)
        after_mask |= in_after.values

# Aplicar BEFORE/AFTER (sem sobrescrever DURING)
df.loc[before_mask & (df["state"] == "NORMAL"), "state"] = "BEFORE"
df.loc[after_mask  & (df["state"] == "NORMAL"), "state"] = "AFTER"

# -----------------------------
# 6) Validações de consistência
# -----------------------------

# Checar conflitos (não deve haver DURING junto com BEFORE/AFTER, por construção)
conf_before_during = int((before_mask & during_mask).sum())
conf_after_during  = int((after_mask & during_mask).sum())

log(f"Conflitos BEFORE∩DURING: {conf_before_during}")
log(f"Conflitos AFTER∩DURING:  {conf_after_during}")

# Distribuição final
dist = df["state"].value_counts(dropna=False).to_dict()
log(f"Distribuição de estados: {dist}")

# Sanidade: todo episódio deve ter ao menos 1 janela DURING
episodes_without_during = 0
for _, ep in episodes.iterrows():
    s = int(ep["start_bucket"])
    e = int(ep["end_bucket"])
    hits = int(((df["bucket_id"] >= s) & (df["bucket_id"] <= e) & (df["state"] == "DURING")).sum())
    if hits == 0:
        episodes_without_during += 1

if episodes_without_during > 0:
    log(f"Aviso: episódios sem janelas DURING após rotulagem: {episodes_without_during}")
else:
    log("OK: todos os episódios têm ao menos uma janela DURING.")

# -----------------------------
# 7) Persistência
# -----------------------------

OUT_FILE = FEATURES_PATH / "window_5min_labeled.parquet"
df.to_parquet(OUT_FILE, compression="snappy", index=False)

summary = {
    "rows": int(len(df)),
    "k_before": int(K_BEFORE),
    "k_after": int(K_AFTER),
    "states_distribution": {k: int(v) for k, v in dist.items()},
    "episodes_total": int(len(episodes)),
    "episodes_without_during": int(episodes_without_during),
    "conflicts_before_during": int(conf_before_during),
    "conflicts_after_during": int(conf_after_during),
}

summary_file = REPORTS_PATH / "06_labeling_states_summary.json"
summary_file.write_text(json.dumps(summary, indent=2, ensure_ascii=False))

log("Notebook 06 finalizado com sucesso.")
df.head()

Mounted at /content/drive
[Bootstrap] Atualizando repositório (git pull).
[Bootstrap] CWD = /content/drive/MyDrive/Mestrado/PPCOMP_DM
FEATURES_PATH = /content/drive/MyDrive/Mestrado/02-datasets/03-features
REPORTS_PATH  = /content/drive/MyDrive/Mestrado/04-reports
[06_labeling_states] Features: shape=(8914, 26)
[06_labeling_states] Episodes: shape=(111, 11)
[06_labeling_states] DURING: janelas marcadas (com possíveis sobreposições por episódio): 200
[06_labeling_states] Conflitos BEFORE∩DURING: 0
[06_labeling_states] Conflitos AFTER∩DURING:  0
[06_labeling_states] Distribuição de estados: {'NORMAL': 7201, 'BEFORE': 854, 'AFTER': 659, 'DURING': 200}
[06_labeling_states] OK: todos os episódios têm ao menos uma janela DURING.
[06_labeling_states] Notebook 06 finalizado com sucesso.


,bucket_id,bucket_start_us,n_events,n_failed,n_machines,n_collections,mean_priority,mean_req_cpus,mean_req_mem,req_cpus_presence_rate,...,event_KILL_count,is_critical,lag_1,lag_2,lag_3,rolling_mean_1h,rolling_std_1h,pct_change,zscore_global,state
0,15,4500000000,44,10,41,29,249.522727,0.008850,0.003220,1.0,...,0,0,11.0,13.0,6.0,10.000000,2.943920,-0.090909,0.416312,NORMAL
1,16,4800000000,22,5,22,19,219.045455,0.011749,0.003593,1.0,...,1,0,10.0,11.0,13.0,9.000000,3.391165,-0.500000,-0.135315,NORMAL
2,17,5100000000,28,5,27,22,279.392857,0.010309,0.004613,1.0,...,0,0,5.0,10.0,11.0,8.333333,3.444803,0.000000,-0.135315,NORMAL
3,18,5400000000,29,7,29,22,259.517241,0.007249,0.009697,1.0,...,2,0,5.0,5.0,10.0,8.142857,3.184785,0.400000,0.085336,NORMAL
4,19,5700000000,36,9,34,27,237.027778,0.010176,0.009284,1.0,...,0,0,7.0,5.0,5.0,8.250000,2.964071,0.285714,0.305987,NORMAL


## Achados do Notebook 06 — Rotulagem Supervisionada

### Estrutura do dataset rotulado

- Total de janelas: 8914
- Total de episódios detectados: 111
- Nenhum episódio ficou sem janela DURING associada.
- Não houve conflitos entre DURING e BEFORE/AFTER.

A precedência definida (DURING > BEFORE/AFTER > NORMAL) foi aplicada corretamente.

---

### Distribuição final das classes

Distribuição de estados:

- NORMAL: 7201
- BEFORE: 854
- AFTER: 659
- DURING: 200

Observações:

- DURING representa aproximadamente 2,24% da base.
- BEFORE e AFTER ampliam o conjunto de estados relevantes,
  permitindo modelar o comportamento pré e pós-degradação.
- O problema deixa de ser puramente binário e passa a permitir
  classificação multiclasse (NORMAL / BEFORE / DURING / AFTER).

---

### Consistência temporal

- A rotulagem respeita integralmente o eixo bucket_id.
- Não há vazamento de informação futura na definição de BEFORE.
- AFTER representa janelas de estabilização pós-evento,
  úteis para análise de transição de regime.

A causalidade temporal foi preservada.

---

### Estrutura supervisionada final

A coluna `state` formaliza o ground truth supervisionado
e permite dois cenários experimentais futuros:

1. Classificação binária:
   - NORMAL vs (BEFORE + DURING + AFTER)
   - NORMAL vs DURING

2. Classificação multiclasse:
   - NORMAL
   - BEFORE
   - DURING
   - AFTER

Essa flexibilidade amplia o potencial analítico do modelo.

---

### Considerações metodológicas

A estratégia adotada:

- Mantém reprodutibilidade
- Preserva granularidade de 5 minutos
- Evita conflitos entre rótulos
- Permite controle explícito do horizonte de contexto (K_BEFORE e K_AFTER)

O dataset `window_5min_labeled.parquet` consolida a transição entre
detecção estatística e modelagem supervisionada estruturada.

---

### Conclusão

O Notebook 06 estabelece formalmente o problema de classificação
temporal em ambientes distribuídos, estruturando a base de estados
operacionais que sustentará os experimentos supervisionados do
Notebook 07 (baseline Random Forest).